# Ship Detection - Tristan Donzé & Liam Loughman

## Abstract 

This notebook contains both the code implementation and report for a ship detection project inspired by the research paper "Optimal Target Detection Using One Channel SAR Complex Imagery" by Armand Lopès, Jérôme Bruniquel, Franck Sery, Jean-Claude Souyris, and Frédéric Adragna. The project focuses on detecting ships in Sentinel-1 SAR imagery using statistical methods. By implementing a combination of Statistical Whitening Filter (SWF) and Likelihood Ratio Test (LRT), followed by radiometric thresholding and morphological operations, we developed a robust non-learning approach for maritime vessel detection. Our methodology was applied to images from the Strait of Malacca, demonstrating effective ship detection capabilities in various maritime conditions.

### Question 1

**Optical Sensors**:

- Advantages: High spatial resolution; rich spectral information useful for identifying material properties; effective in clear weather and daylight.

- Drawbacks: Limited usability in cloudy or night conditions; atmospheric distortion can affect data quality.

**SAR Sensors**:

- Advantages: Operates day and night; penetrates clouds and adverse weather conditions; provides structural and geometric information.

- Drawbacks: Lower spatial resolution compared to optical sensors; speckle noise complicates image processing; requires advanced preprocessing for analysis.

**LiDAR**:

- Advantages: Provides precise 3D topographic information; effective for height and structure analysis.

- Drawbacks: Expensive; less effective in wide-area maritime applications compared to SAR; weather dependence due to laser attenuation.

**SAR Altimeter**:

- Advantages: Measures precise sea surface heights; useful for detecting wakes and identifying the maritime environment.

- Drawbacks: Limited spatial resolution; not specifically designed for detecting small targets like ships.


### Question 2

**Sensor Selection:**
SAR sensors were exclusively used in this project, reflecting their robustness for ship detection regardless of weather and illumination conditions. No multimodal approach was used.

### Question 3

**Image Source:**

Open-access HR SAR images from Sentinel-1 from the straight of Malacca were used.

### Question 4

Non-Learning Approach Used:

The project relied on statistical and radiometric methods for ship detection:

1. Spatial Whitening Filter (SWF): Preprocessing to normalize background clutter by computing mean and standard deviation of the background regions.

2. Likelihood Ratio Test (LRT): A statistical approach to distinguish between background and target regions by comparing radiometric characteristics.

3. Radiometric Thresholding: Setting thresholds based on desired False Alarm Rates (FAR) to isolate ships from the background clutter.

4. Morphological Operations: Applied for postprocessing, such as removing small objects and filling holes, to refine ship detections.

## Data Preprocessing: 

### Download the Image

The original data was downloaded from the [Copernicus Browser](https://browser.dataspace.copernicus.eu). You can find it by searching for ```S1A_IW_SLC__1SDV_20241228T113437_20241228T113505_057192_070894_70FE``` in the search bar.

<img src="image.png" alt="Straight of Malacca" width="700"/>

### Preprocessing
We then used the SNAP software (version 9.0.0) to preprocess the image. After opening the zip file, we applied the following steps: 
- **Radiometric Calibration**: This step converts pixel values into radar reflectivity values (sigma naught, σ°). To reproduce this in SNAP, use the tool ```Radar > Radiometric > Calibration``` and check only the "Output sigma0 band" option.

- **Creating an Intensity Image**: The real (i) and imaginary (q) components are combined to obtain the signal amplitude, which is less sensitive to speckle than individual i and q values. To reproduce this in SNAP, use the tool ```Raster > Band Maths```, and input the expression `sqrt(Sigma0_IW1_VV)` followed by `sqrt(Sigma0_IW2_VV)`. (We used the VV channel, which provides better contrast for detecting ships because it is more sensitive to vertical reflective structures and metallic surfaces.)

- **Exporting the Images**: Finally, we exported the images in `.tif` format. To reproduce this in SNAP, click on ```File > Export > Other > View as Image > Image Region: Full Scene, File Format: Tiff - Tagged Image File Format```.

The original image of the Strait of Malacca was divided into three sub-images arranged left to right. Since the right sub-image mostly consists of land, we will focus only on the left and center sub-images.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from scipy.stats import norm
from skimage import morphology
from skimage.measure import label, regionprops
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
from shapely.geometry import box, MultiPolygon
from shapely.ops import unary_union

### Loading Preprocessed Images

We begin by loading the preprocessed .tif images using rasterio. This allows us to access the image data and its metadata for further analysis.

In [2]:
file_paths = [
    'S1A_IW1_SLC_Cal_vv_intensity_VV_IW1.tif',
    'S1A_IW1_SLC_Cal_vv_intensity_VV_IW2.tif',
]

In [ ]:
image_data = {}

for path in file_paths:
    with rasterio.open(path) as src:
        image = src.read(1)
        image_data[path] = {
            'data': image,
            'profile': src.profile
        }
        print(f"Loaded {path} with shape {image.shape} and dtype {image.dtype}")

### Visualizing Preprocessed Images

To understand the data better, we visualize the preprocessed images. This helps in identifying features and potential areas of interest, such as ships.

The below code will display grayscale images of the preprocessed data with amplitude values in decibels (dB).

In [ ]:
def visualize_image(image, title):
    plt.figure(figsize=(10, 8))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.colorbar(label='Amplitude (dB)')
    plt.tight_layout()
    plt.show()

for path, data in image_data.items():
    visualize_image(data['data'], f"Preprocessed Image\n{path}")

### Statistical Whitening Filter (SWF)

#### Selecting Background Regions

To enhance the detection of ships, we first identify background regions in the image. These regions are typically areas with minimal activity, such as open water, and are used to estimate background statistics.

In [ ]:
def visualize_mask(mask, title):
    plt.figure(figsize=(10, 8))
    mask_uint8 = mask.astype(np.uint8)
    plt.imshow(mask_uint8, cmap='gray')
    plt.title(title)
    plt.colorbar(label='Mask Value')
    plt.tight_layout()
    plt.show()

def select_background_region(image, percentile=5):
    thresh = np.percentile(image, percentile)
    background_mask = image <= thresh
    return background_mask

for path, data in image_data.items():
    image = data['data']
    background_mask = select_background_region(image, percentile=15)
    data['background_mask'] = background_mask
    
    visualize_mask(background_mask, f"Background Mask - {path}")

The background masks highlight areas considered as background based on the specified percentile threshold.

#### Applying Statistical Whitening Filter (SWF)

Statistical Whitening Filter normalizes the image by subtracting the mean and dividing by the standard deviation of the background. This enhances the contrast between foreground objects (ships) and the background.

In [ ]:
def apply_swf(image, mean_bg, std_bg):
    whitened = (image - mean_bg) / std_bg
    return whitened

def estimate_background_stats(image, background_mask):
    background_pixels = image[background_mask]
    mean_bg = np.mean(background_pixels)
    std_bg = np.std(background_pixels)
    return mean_bg, std_bg

for path, data in image_data.items():
    image = data['data']
    
    mean_bg, std_bg = estimate_background_stats(image, data['background_mask'])

    data['mean_bg'] = mean_bg
    data['std_bg'] = std_bg 
    
    whitened_image = apply_swf(image, mean_bg, std_bg)
    data['whitened'] = whitened_image

    visualize_image(whitened_image, f"Whitened Image - {path}")

Whitened images display normalized values, making it easier to identify anomalies such as ships.

### Likelihood Ratio Test (LRT)

#### Computing LRT Statistics

The Likelihood Ratio Test (LRT) is used to distinguish between background and foreground (ship) pixels based on their statistical properties.

In [ ]:
def compute_lrt_statistic(whitened_image, mean_bg, std_bg, mean_fg, std_fg):
    log_term = np.log(std_bg / std_fg)
    bg_term = ((whitened_image - mean_bg) ** 2) / (2 * std_bg ** 2)
    fg_term = ((whitened_image - mean_fg) ** 2) / (2 * std_fg ** 2)
    lrt_stat = log_term + bg_term - fg_term
    return lrt_stat

for path, data in image_data.items():
    foreground_mask = ~data['background_mask']
    image = data['data']
    whitened_image = data['whitened']
    data['mean_fg'] = np.mean(image[foreground_mask])
    data['std_fg'] = np.std(image[foreground_mask])
    print(data['mean_bg'], data['std_bg'], data['mean_fg'], data['std_fg'])
    lrt_stat = compute_lrt_statistic(whitened_image, data['mean_bg'], data['std_bg'], data['mean_fg'], data['std_fg'])
    data['lrt_stat'] = lrt_stat
    
    visualize_image(lrt_stat, f"LRT Statistic - {path}")

The LRT statistic image emphasizes regions where the likelihood of being a ship is higher.

#### Normalizing LRT Statistics

Normalization scales the LRT statistics to a standard range, facilitating thresholding and subsequent detection steps.

In [ ]:
def normalize_lrt(lrt_image):
    mean_lrt = np.mean(lrt_image)
    std_lrt = np.std(lrt_image)
    normalized_lrt = (lrt_image - mean_lrt) / std_lrt
    return normalized_lrt, mean_lrt, std_lrt

for path, data in image_data.items():
    lrt_image = data['lrt_stat']  # Replace with your computed LRT statistics
    normalized_lrt, mean_lrt, std_lrt = normalize_lrt(lrt_image)
    data['normalized_lrt'] = normalized_lrt
    
    print(f"{path}: Mean (before) = {np.mean(lrt_image):.4f}, Std (before) = {np.std(lrt_image):.4f}")
    print(f"{path}: Mean (normalized) = {np.mean(normalized_lrt):.4f}, Std (normalized) = {np.std(normalized_lrt):.4f}")
    
    plt.figure(figsize=(10, 8))
    plt.imshow(normalized_lrt, cmap='gray', norm=Normalize(vmin=0, vmax=1))
    plt.title(f"Normalized LRT - {path}")
    plt.colorbar(label='Normalized LRT Value')
    plt.tight_layout()
    plt.show()

Normalized LRT images have a mean of 0 and a standard deviation of 1, making it easier to apply consistent thresholds across images.

### Radiometric Thresholding

#### Computing Thresholds Based on False Alarm Rate (FAR)

Thresholding helps in distinguishing between true detections (ships) and false alarms. We compute thresholds corresponding to specific FARs using the standard normal distribution.

In [ ]:
def compute_threshold(FAR):
    threshold = norm.ppf(1 - FAR)
    return threshold

FAR = [3e-1, 1e-1]

thresholds = {}

for i, path in enumerate(file_paths):    
    threshold = compute_threshold(FAR[i])
    print(f"Threshold for {path} at FAR {FAR[i]}: {threshold:.4f}")
    thresholds[path] = threshold

for path, data in image_data.items():
    normalized_lrt = data['normalized_lrt']
    plt.figure(figsize=(10, 6))
    plt.hist(normalized_lrt.flatten(), bins=100, color='blue', alpha=0.7)
    plt.title(f"LRT Histogram - {path}")
    plt.xlabel("LRT Statistic")
    plt.ylabel("Frequency")
    plt.axvline(thresholds[path], color='red', linestyle='dashed', linewidth=2, label='Threshold')
    plt.legend()
    plt.show()

Histograms display the distribution of normalized LRT values with the computed thresholds indicated by red dashed lines.

#### Applying Thresholds to Detect Ships

Using the computed thresholds, we classify pixels as detections (potential ships) or non-detections.

In [ ]:
def apply_threshold(lrt_stat, threshold):
    detections = lrt_stat > threshold
    return detections

for path, data in image_data.items():
    normalized_lrt = data['normalized_lrt']
    detections = apply_threshold(normalized_lrt, thresholds[path])
    data['detections'] = detections
    
    visualize_image(detections, f"Detections (FAR={FAR}) - {path}")

Detection masks highlight areas where the LRT statistic exceeds the threshold, indicating potential ship locations.

### Cleaning Detections

#### Removing Small Artifacts

To improve detection accuracy, we remove small objects and fill small holes within the detection masks. This helps in eliminating noise and insignificant detections.

In [ ]:
def clean_detections(detections, min_size=20, area_threshold=20):
    detections_clean = morphology.remove_small_holes(detections, area_threshold=area_threshold)
    detections_clean = morphology.remove_small_objects(detections_clean, min_size=min_size)
    return detections_clean

for path, data in image_data.items():
    detections = data['detections']
    detections[data['background_mask']] = 0
    detections_clean = clean_detections(detections, min_size=100, area_threshold=1000)
    data['detections_clean'] = detections_clean
    
    visualize_image(detections_clean, f"Cleaned Detections - {path}")

Cleaned detection masks are more refined, showing fewer false positives and clearer ship outlines.

### Initial Counting of Detected Ships

### Identifying Connected Components

We count the number of detected ships by identifying connected regions in the cleaned detection masks. Each connected region is assumed to correspond to an individual ship.

In [ ]:
def count_ships(detections_clean):
    labeled_detections = label(detections_clean, connectivity=2)
    regions = regionprops(labeled_detections)
    num_ships = len(regions)
    return num_ships, regions

for path, data in image_data.items():
    detections_clean = data['detections_clean']
    num_ships, regions = count_ships(detections_clean)
    data['num_ships'] = num_ships
    print(f"Number of detected ships in {path}: {num_ships}")

### Visualizing Detections with Bounding Boxes

#### Drawing Bounding Boxes Around Ships

To visualize the detected ships, we draw bounding boxes around each detected region on the original image.

In [ ]:
def draw_bounding_boxes(original_image, regions, title):
    plt.figure(figsize=(10, 8))
    plt.imshow(original_image, cmap='gray')
    plt.title(title)
    plt.colorbar(label='Amplitude (dB)')
    
    ax = plt.gca()
    for region in regions:
        minr, minc, maxr, maxc = region.bbox
        rect = Rectangle((minc, minr), maxc - minc, maxr - minr,
                         linewidth=1, edgecolor='yellow', facecolor='none')
        ax.add_patch(rect)
    
    plt.tight_layout()
    plt.show()

for path, data in image_data.items():
    original_image = data['data']
    detections_clean = data['detections_clean']
    num_ships = data['num_ships']
    regions = label(detections_clean, connectivity=2)
    props = regionprops(regions)
    title = f"Detected Ships ({num_ships}) - {path}"
    draw_bounding_boxes(original_image, props, title)

### Refining Detections

#### Merging Bounding Boxes Based on Proximity

To handle cases where a single ship might be detected as multiple regions, we merge bounding boxes that are in close proximity.

In [ ]:
def merge_bounding_boxes(detections_clean, proximity_threshold=20):
    
    labeled = label(detections_clean)
    regions = regionprops(labeled)
    
    boxes = [box(region.bbox[1], region.bbox[0], region.bbox[3], region.bbox[2]) for region in regions]
    
    if not boxes:
        return detections_clean.copy()
    
    buffered_boxes = [b.buffer(proximity_threshold) for b in boxes]
    
    merged = unary_union(buffered_boxes)
    
    refined_detections = np.zeros_like(detections_clean, dtype=bool)
    
    if isinstance(merged, MultiPolygon):
        geometries = merged.geoms
    elif merged.geom_type == 'Polygon':
        geometries = [merged]
    else:
        geometries = [] 

    for geom in geometries:
        minx, miny, maxx, maxy = geom.bounds
        min_row, min_col = int(np.floor(miny)), int(np.floor(minx))
        max_row, max_col = int(np.ceil(maxy)), int(np.ceil(maxx))
        
        min_row = max(min_row, 0)
        min_col = max(min_col, 0)
        max_row = min(max_row, detections_clean.shape[0])
        max_col = min(max_col, detections_clean.shape[1])
        
        refined_detections[min_row:max_row, min_col:max_col] = 1

    return refined_detections

for path, data in image_data.items():
    detections_clean = data['detections_clean']
    refined_detections = merge_bounding_boxes(detections_clean, proximity_threshold=30) 
    data['refined_detections'] = refined_detections

    num_ships, regions = count_ships(refined_detections)
    data['num_ships_refined'] = num_ships
    print(f"Number of detected ships after first refinement in {path}: {num_ships}")
    
    labeled_refined = label(refined_detections, connectivity=2)
    props_refined = regionprops(labeled_refined)
    draw_bounding_boxes(data['data'], props_refined, f"First Refined Detected Ships ({num_ships}) - {path}")

Refined detections reduce duplicate counts and ensure each ship is represented by a single bounding box.

#### Filtering Detections by Size

To further enhance detection accuracy, we filter out detections that are either too small or too large to be ships based on area constraints.

In [ ]:

def filter_detections_by_size(detections_clean, max_area=5000):
    
    labeled = label(detections_clean, connectivity=2)
    regions = regionprops(labeled)
    
    filtered_detections = np.zeros_like(detections_clean, dtype=bool)
    
    for region in regions:
        area = region.area
        min_row, min_col, max_row, max_col = region.bbox
        
        if area <= max_area:
            filtered_detections[min_row:max_row, min_col:max_col] = 1
        else:
            print(f"Removed detection at [(Row: {min_row}, Col: {min_col}) to (Row: {max_row}, Col: {max_col})] "
                  f"with Area: {area}")
    
    return filtered_detections

for path, data in image_data.items():
    refined_detections = data['refined_detections']
    
    filtered_detections = filter_detections_by_size(refined_detections, max_area=16000)
    data['filtered_detections'] = filtered_detections
    
    num_ships_filtered, regions_filtered = count_ships(filtered_detections)
    data['num_ships_filtered'] = num_ships_filtered
    print(f"Number of detected ships after size filtering in {path}: {num_ships_filtered}")
    
    labeled_filtered = label(filtered_detections, connectivity=2)
    props_filtered = regionprops(labeled_filtered)
    draw_bounding_boxes(data['data'], props_filtered, f"Filtered Detected Ships ({num_ships_filtered}) - {path}")

Size filtering removes unlikely ship detections based on predefined area thresholds, resulting in a more accurate ship count.

## Conclusion

Some ships were not correctly detected, and certain land areas (primarily islands) were mistakenly identified as ships. Additionally, we only used two images, so the approach was not fully generalized to all possible cases. Nevertheless, we are satisfied with the results obtained, as the majority of the ships were successfully detected.